# granite.materials.smi-SSED - Encoder & Decoder

In [1]:
import sys
sys.path.append('../inference')

In [2]:
# materials.smi-ssed (smi-ssed)
from smi_ssed.load import load_smi_ssed

# Data
import pandas as pd
import numpy as np
import torch

# Chemistry
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
from rdkit.DataStructs import FingerprintSimilarity
from rdkit.DataStructs import TanimotoSimilarity

/u/vshirasuna/miniforge3/envs/finetune/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# function to canonicalize SMILES
def normalize_smiles(smi, canonical=True, isomeric=False):
    try:
        normalized = Chem.MolToSmiles(
            Chem.MolFromSmiles(smi), canonical=canonical, isomericSmiles=isomeric
        )
    except:
        normalized = None
    return normalized

# function to calculate pairwise Tanimoto similarity
def calculate_tanimoto_similarities(fps1, fps2):
    similarities = []
    for i in range(len(fps1)):
            sim = TanimotoSimilarity(fps1[i], fps2[i])
            similarities.append(sim)
    return similarities

## Load smi-ssed

In [4]:
model_smi_ssed = load_smi_ssed(
    folder='../inference/smi_ssed',
    ckpt_filename='smi-ssed_130.pt'
)

/u/vshirasuna/miniforge3/envs/finetune/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Random Seed: 12345
Vocab size: 2393
[INFERENCE MODE - smi-ssed]


## Load Dataset

In [5]:
df_moses = pd.read_csv("./data/moses_test.csv", nrows=1000)

In [6]:
df_moses['SMILES'] = df_moses['SMILES'].apply(normalize_smiles)
df_test_normalized = df_moses.dropna()
print(df_test_normalized.shape)
df_test_normalized.head()

(1000, 1)


,SMILES
0,CC1C2CCC(C2)C1CN(CCO)C(=O)c1ccc(Cl)cc1
1,COc1ccc(-c2cc(=O)c3c(O)c(OC)c(OC)cc3o2)cc1O
2,CCOC(=O)c1ncn2c1CN(C)C(=O)c1cc(F)ccc1-2
3,Clc1ccccc1-c1nc(-c2ccncc2)no1
4,CC(C)(Oc1ccc(Cl)cc1)C(=O)OCc1cccc(CO)n1


## Encode SMILES - smi-ssed

In [7]:
with torch.no_grad():
    encode_embeddings = model_smi_ssed.encode(df_moses['SMILES'], return_torch=True)

/u/vshirasuna/miniforge3/envs/finetune/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.07it/s]


## Decode smi-ssed embeddings into SMILES

In [8]:
with torch.no_grad():
    decoded_smiles = model_smi_ssed.decode(encode_embeddings)

In [9]:
decoded_smiles[0:5]

['CC1C2CCC(C1)C1CN(CCO)C(=O)c1ccc(Cl)cc1',
 'COc1ccc(-c2cc(=O)c3c(O)c(OC)c(OC)cc3o2)cc1O',
 'CCOC(=O)c1ncn2c1CN(C)C(=O)c1cc(F)ccc1-2',
 'Clc1ccccc1-c1nc(-c2ccncc2)no1',
 'CC(C)(Oc1ccc(Cl)cc1)C(=O)OCc1cccc(CO)n1']

## Compare similarities

In [10]:
# Convert SMILES to RDKit molecule objects
mols1 = [Chem.MolFromSmiles(smiles) for smiles in df_moses['SMILES'].to_list()]
mols2 = [Chem.MolFromSmiles(smiles) for smiles in decoded_smiles]

# Filter null molecules
null_mask = [False if value == None else True for value in mols2]
filtered_mols1 = [m for (m, c) in zip(mols1, null_mask) if c]
filtered_mols2 = [m for (m, c) in zip(mols2, null_mask) if c]

# Compute fingerprints for each molecule
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)
fps1 = [mfpgen.GetFingerprint(mol) for mol in filtered_mols1]
fps2 = [mfpgen.GetFingerprint(mol) for mol in filtered_mols2]

# Calculate Tanimoto similarities
tanimoto_similarities = calculate_tanimoto_similarities(fps1, fps2)

# Append null molecules (=0)
n_null = sum([not v for v in null_mask])
tanimoto_similarities = tanimoto_similarities + ([0]*n_null)

# Calculate the mean similarity
mean_similarity = np.mean(tanimoto_similarities)

# Print the mean similarity
print(f"Mean Tanimoto Similarity: {mean_similarity:.2f}")

[15:12:38] SMILES Parse Error: unclosed ring for input: 'CC1C2CCC(C1)C1CN(CCO)C(=O)c1ccc(Cl)cc1'
[15:12:38] Can't kekulize mol.  Unkekulized atoms: 11 12 13 14 15 17 18
[15:12:38] SMILES Parse Error: extra close parentheses while parsing: COc1ccc(C2c3ccnoc3C)CC(C)(O)C2C(C)=O)cc1
[15:12:38] SMILES Parse Error: Failed parsing SMILES 'COc1ccc(C2c3ccnoc3C)CC(C)(O)C2C(C)=O)cc1' for input: 'COc1ccc(C2c3ccnoc3C)CC(C)(O)C2C(C)=O)cc1'


Mean Tanimoto Similarity: 1.00
